In [3]:
import cfbd
import requests
import json
from itertools import islice
import time
from cfbd.rest import ApiException
from pprint import pprint
import sys, subprocess
import networkx as nx
from config import API_KEY

In [4]:
# access api

configuration = cfbd.Configuration(
    access_token = API_KEY
)

In [11]:
# getting team data to cross ref
years = [2021, 2022, 2023, 2024, 2025]

# This is the only dictionary you need to create here.
node_attributes_by_year = {}

for year in years:
    print(f"Fetching all team data for attributes for {year}...")
    teams_api = cfbd.TeamsApi(cfbd.ApiClient(configuration))
    all_teams = teams_api.get_teams(year=year)
    print(f"Found {len(all_teams)} teams.")
    
    current_year_attrs = {}
    for team in all_teams:
        current_year_attrs[team.school] = {
            # Use 'Unknown' as a default string if data is None
            'classification': str(team.classification) if team.classification else 'Unknown',
            'conference': str(team.conference) if team.conference else 'Unknown',
            # Use 0.0 as a default float if data is None
            'latitude': float(team.location.latitude) if team.location and team.location.latitude else 0.0,
            'longitude': float(team.location.longitude) if team.location and team.location.longitude else 0.0
        }
    node_attributes_by_year[year] = current_year_attrs

print("Node attribute maps created successfully.")

Fetching all team data for attributes for 2021...
Found 671 teams.
Fetching all team data for attributes for 2022...
Found 673 teams.
Fetching all team data for attributes for 2023...
Found 674 teams.
Fetching all team data for attributes for 2024...
Found 681 teams.
Fetching all team data for attributes for 2025...
Found 683 teams.
Node attribute maps created successfully.


In [6]:
# getting transfer portal data

# Enter a context with an instance of the API client
with cfbd.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    api_instance = cfbd.PlayersApi(api_client)

api_response_2025 = api_instance.get_transfer_portal(year=2025)
api_response_2024 = api_instance.get_transfer_portal(year=2024)
api_response_2023 = api_instance.get_transfer_portal(year=2023)
api_response_2022 = api_instance.get_transfer_portal(year=2022)
api_response_2021 = api_instance.get_transfer_portal(year=2021)


In [12]:
# convert to graphml

data_by_year = {
    2025: api_response_2025,
    2024: api_response_2024,
    2023: api_response_2023,
    2022: api_response_2022,
    2021: api_response_2021
}

node_attr_map = node_attributes_by_year

default_attrs = {
    'classification': 'Unknown',
    'conference': 'Unknown',
    'latitude': 0.0,
    'longitude': 0.0
}

# Add nodes and edges to the graph G based on data
# Nodes: school names (origin and destination)
# Edges: a directed edge origin -> destination per player; edge attributes aggregate players
for year, data in data_by_year.items():
    G = nx.DiGraph()
    print(f"Processing data for {year}...")
    for t in data:
        origin = t.origin.strip() if getattr(t, 'origin', None) else None
        dest = t.destination.strip() if getattr(t, 'destination', None) else None

        # add nodes if present
        if origin:
            G.add_node(origin)
        if dest:
            G.add_node(dest)

        # only create edges when both origin and destination exist
        if origin and dest:
            player = f"{getattr(t, 'first_name', '')} {getattr(t, 'last_name', '')}".strip()
            pos = getattr(t, 'position', None)
            date = getattr(t, 'transfer_date', None)
            date_iso = date.isoformat() if date is not None else None
            rating = getattr(t, 'rating', None)
            stars = getattr(t, 'stars', None)
            eligibility = getattr(t, 'eligibility', None)

            if G.has_edge(origin, dest):
                edge = G[origin][dest]
                edge.setdefault('players', []).append(player)
                edge.setdefault('positions', []).append(pos)
                edge.setdefault('dates', []).append(date_iso)
                edge.setdefault('ratings', []).append(rating)
                edge.setdefault('stars', []).append(stars)
                edge.setdefault('eligibility', []).append(str(eligibility))
                edge['weight'] = edge.get('weight', 1) + 1
            else:
                G.add_edge(origin, dest, players=[player], positions=[pos], dates=[date_iso], ratings=[rating], stars=[stars], eligibility=[str(eligibility)], weight=1)


    # Serialize any list-valued (or None) edge attributes to strings for GraphML compatibility
    for u, v, attrs in G.edges(data=True):
        for k in list(attrs.keys()):
            val = attrs[k]
            if isinstance(val, list):
                # join list elements into a single string; convert None -> empty string
                attrs[k] = ' | '.join(['' if x is None else str(x) for x in val])
            elif val is None:
                attrs[k] = ''

    #Get the map of attributes for the specific year
    current_year_node_attrs = node_attr_map.get(year, {}) # Get the map, or an empty dict

    # Create the final mapping, applying defaults to any node not in the map
    final_attrs_for_graph = {
        node: current_year_node_attrs.get(node, default_attrs) for node in G.nodes()
    }

    # Set all attributes at once. 
    # NetworkX unpacks the inner dictionaries automatically.
    nx.set_node_attributes(G, final_attrs_for_graph)

    # --- 4. VERIFY AND SAVE ---
    print("\n--- Verification ---")
    # Check the attributes for a few schools
    for school in ['Alabama', 'North Dakota State', 'Ohio State']:
        if school in G:
            print(f"School: {school}, Attributes: {G.nodes[school]}")

    filename = f"transfer_portal_{year}.graphml"

    # write graphml
    nx.write_graphml(G, filename)
    print(f"Wrote {len(G.nodes())} nodes and {len(G.edges())} edges to {filename}")

Processing data for 2025...

--- Verification ---
School: Alabama, Attributes: {'classification': 'fbs', 'conference': 'SEC', 'latitude': 33.2082752, 'longitude': -87.5503836}
School: North Dakota State, Attributes: {'classification': 'fcs', 'conference': 'MVFC', 'latitude': 46.9029938, 'longitude': -96.8003251}
School: Ohio State, Attributes: {'classification': 'fbs', 'conference': 'Big Ten', 'latitude': 40.0016447, 'longitude': -83.0197266}
Wrote 410 nodes and 3296 edges to transfer_portal_2025.graphml
Processing data for 2024...

--- Verification ---
School: Alabama, Attributes: {'classification': 'fbs', 'conference': 'SEC', 'latitude': 33.2082752, 'longitude': -87.5503836}
School: North Dakota State, Attributes: {'classification': 'fcs', 'conference': 'MVFC', 'latitude': 46.9029938, 'longitude': -96.8003251}
School: Ohio State, Attributes: {'classification': 'fbs', 'conference': 'Big Ten', 'latitude': 40.0016447, 'longitude': -83.0197266}
Wrote 352 nodes and 2340 edges to transfer_